## Linear model testing

In [1]:
# Now we will do what we had to do in the training of linear model 


In [2]:
import numpy as np
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt

In [3]:
X=pd.read_csv('selected_x_train.csv')
y=pd.read_csv('y_train_final.csv')

In [4]:
y=y.iloc[:,1:]
y.head()

,SalePrice
0,12.247694
1,12.109011
2,12.317167
3,11.849398
4,12.429216


In [5]:
def rmse_cv(model):
    rmse = np.sqrt(-cross_val_score(model, X, y, scoring="neg_mean_squared_error", cv=5)).mean()
    return rmse
def evaluation(y_hat, predictions):
    mae = mean_absolute_error(y_hat, predictions)
    mse = mean_squared_error(y_hat, predictions)
    rmse = np.sqrt(mean_squared_error(y_hat, predictions))
    r_squared = r2_score(y_hat, predictions)
    return mae, mse, rmse, r_squared

In [6]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test=train_test_split(X,y,random_state=0,test_size=0.33)

In [7]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

ridge=Ridge()
ridge.fit(x_train,y_train)

y_predicted=ridge.predict(x_test)
mae, mse, rmse, r_squared=evaluation(y_predicted,y_test)
ridge_score={'model':'ridge',
             'RMSE':rmse,
             'R2':r_squared,
             'true_error':mae}
ridge_score

{'model': 'ridge',
 'RMSE': 0.13368507311820418,
 'R2': 0.8787706753826393,
 'true_error': 0.09168355502664106}

Lets try to improve the ridge score 

In [8]:
from sklearn.model_selection import GridSearchCV


In [9]:
grid_values={'alpha':[0.0001,0.001,0.005,0.01,0.05,0.1,1],
             'max_iter':[100,400,500,1000],'solver':['auto','svd']}
ridge_modified_scores=GridSearchCV(ridge,param_grid=grid_values,scoring="neg_mean_squared_error",verbose=0)
ridge_modified_scores.fit(x_train,y_train)


GridSearchCV(estimator=Ridge(),
             param_grid={'alpha': [0.0001, 0.001, 0.005, 0.01, 0.05, 0.1, 1],
                         'max_iter': [100, 400, 500, 1000],
                         'solver': ['auto', 'svd']},
             scoring='neg_mean_squared_error')

In [10]:
y_predicted=ridge_modified_scores.predict(x_test)
mae, mse, rmse, r_squared=evaluation(y_predicted,y_test)
ridge_score={'model':'ridge',
             'RMSE':rmse,
             'R2':r_squared,
             'true_error':mae}
ridge_score

{'model': 'ridge',
 'RMSE': 0.13542478542866906,
 'R2': 0.8778289740216606,
 'true_error': 0.09284354280922202}

In [11]:
print(ridge_modified_scores.best_params_)
print(ridge_modified_scores.best_score_)

{'alpha': 0.1, 'max_iter': 100, 'solver': 'svd'}
-0.01740886662049383


In [12]:
# Now lets see how Lasoo regression works
from sklearn.linear_model import Lasso
lasso=Lasso()
lasso_param={'alpha':[0.0001,0.0005,0.001,0.005,0.01,0.05,0.1,0.5,1],
             'max_iter':[100,500,1000],
             'warm_start':[True,False]}

lasso_score=GridSearchCV(lasso,lasso_param,scoring="neg_mean_squared_error")
lasso_score.fit(x_train,y_train)
lasso_score.best_params_

c:\users\sekhu\appdata\local\programs\python\python39\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.845824473025409, tolerance: 0.012631920296592377
  model = cd_fast.enet_coordinate_descent(
c:\users\sekhu\appdata\local\programs\python\python39\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.5069520052099268, tolerance: 0.01274798228388607
  model = cd_fast.enet_coordinate_descent(
c:\users\sekhu\appdata\local\programs\python\python39\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.184575397856362, tolerance: 0.013502645983989994
  model = cd_fast.enet_coordinate_descent(
c:\users\sekh

c:\users\sekhu\appdata\local\programs\python\python39\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.04263558896271391, tolerance: 0.012631920296592377
  model = cd_fast.enet_coordinate_descent(
c:\users\sekhu\appdata\local\programs\python\python39\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.11122727040353197, tolerance: 0.013502645983989994
  model = cd_fast.enet_coordinate_descent(
c:\users\sekhu\appdata\local\programs\python\python39\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.04263558896271391, tolerance: 0.012631920296592377
  model = cd_fast.enet_coordinate_descent(
c:\user

{'alpha': 0.0001, 'max_iter': 100, 'warm_start': True}

In [13]:
y_predicted_lasso=lasso_score.predict(x_test)
mae, mse, rmse, r_squared=evaluation(y_predicted_lasso,y_test)
lasso_score={'model':'lasso',
             'RMSE':rmse,
             'R2':r_squared,
             'true_error':mae}
lasso_score

{'model': 'lasso',
 'RMSE': 0.13549565909845582,
 'R2': 0.8774412438333162,
 'true_error': 0.09295903422085608}

In [33]:
from xgboost import XGBRegressor
xgb_parameters={'learning_rate':[0.001,0.01,0.1,1],'n_estimators':[100,500,1000]}
xgb=XGBRegressor()
xgb_score=GridSearchCV(xgb,xgb_parameters,scoring="r2")

xgb_score.fit(x_train, y_train)
predictions = xgb_score.predict(x_test)

mae, mse, rmse, r_squared = evaluation(y_test, predictions)
print("MAE:", mae)
print("MSE:", mse)
print("RMSE:", rmse)
print("R2 Score:", r_squared)
print("-"*30)
rmse_cross_val = rmse_cv(xgb)
print("RMSE Cross-Validation:", rmse_cross_val)

MAE: 0.09116026540959184
MSE: 0.016388951276522845
RMSE: 0.12801933946292196
R2 Score: 0.8916479644606374
------------------------------
RMSE Cross-Validation: 0.14152811436434795


In [29]:
xgb = XGBRegressor(n_estimators=1000, learning_rate=0.01)
xgb.fit(x_train, y_train)
predictions = xgb.predict(x_test)

mae, mse, rmse, r_squared = evaluation(y_test, predictions)
print("MAE:", mae)
print("MSE:", mse)
print("RMSE:", rmse)
print("R2 Score:", r_squared)
print("-"*30)
rmse_cross_val = rmse_cv(xgb)
print("RMSE Cross-Validation:", rmse_cross_val)

MAE: 0.09087052132123054
MSE: 0.01617459331349998
RMSE: 0.12717937456010694
R2 Score: 0.8930651461482094
------------------------------
RMSE Cross-Validation: 0.13148429486619842


In [24]:
y_train.head()

,SalePrice
1127,12.464583
1424,11.877569
587,11.827736
1157,12.345835
938,12.387556


In [ ]:
## you can also use random forest regressor 
## for random forest regressor 
##feature_importances = grid_search.best_estimator_.feature_importances_